In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import torch
from tqdm import tqdm

In [2]:
dat = pd.read_csv('merged_data.csv')
dat = dat.drop(columns=['kWh'])
dat = dat.rename(columns={'Unnamed: 0': 'datetime'})
dat['datetime'] = pd.to_datetime(dat['datetime'])
dat.head()

In [3]:
dat.shape

In [4]:
# Extract day of the week (0 = Monday, 6 = Sunday)
dat['weekday'] = dat['datetime'].dt.weekday
dat['month'] = dat['datetime'].dt.month
dat['time'] = dat['datetime'].dt.time

# Encode 'month' and 'weekday' as categorical variables
dat['month'] = dat['month'].astype('category')
dat['weekday'] = dat['weekday'].astype('category')

In [5]:
# Perform one-hot encoding on 'month' and 'weekday'
df_encoded = pd.get_dummies(dat, columns=['month', 'weekday', "time"], prefix=['month', 'weekday', 'time'])

# remove datetime
df_encoded = df_encoded.drop('datetime', axis=1)

In [6]:
df_encoded

In [7]:
df_encoded.isna().sum()

# Model


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR


In [9]:
# Extract features and labels
labels = df_encoded['MWh'].values
features = df_encoded.drop(['MWh'], axis=1).values

In [10]:
# Normalize features using Min-Max scaling
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [11]:
# Define sequence length (number of time steps for LSTM)
sequence_length = 7*24  # Use the previous 7 days as input, 24 measurements

In [12]:
# Create sequences and labels for training
X, y = [], []
for i in range(len(features) - sequence_length):
    X.append(features[i:i+sequence_length])
    y.append(labels[i+sequence_length])
    
    print(labels[i+sequence_length])

X, y = np.array(X), np.array(y)

In [13]:
# Split data into training and testing sets
split_ratio = 0.8  # Adjust as needed
split_index = int(split_ratio * len(X))


X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [14]:

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [15]:
# Create DataLoader for training and testing
batch_size = 32  # Adjust as needed
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Get the output from the last time step
        return out

In [17]:
input_size = X_train.shape[2]  # Number of features
hidden_size = 132  # Adjust as needed
num_layers = 20  # Adjust as needed
output_size = 1  # Adjust for regression tasks

In [18]:


model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

scheduler = StepLR(optimizer, step_size=10, gamma=0.9)




In [19]:
# Define the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Move the model to the GPU
model.to(device)

# Move data to GPU
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [21]:
# Training loop
model.train()
num_epochs = 100  # Adjust as needed
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    scheduler.step()



In [ ]:

# Rolling window evaluation
model.eval()
predictions = []

for i in range(len(X_test)):
    if i < sequence_length:
        # Initial predictions using the training data
        predictions.append(y_train[i].item())
    else:
        inputs = X_test[i].unsqueeze(0)
        with torch.no_grad():
            output = model(inputs)
        predictions.append(output.item())


        

In [ ]:
# Calculate and print evaluation metrics (e.g., MAE, MSE, etc.)
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Move the predictions tensor from GPU to CPU
predictions = np.array(predictions)
y_test_cpu = y_test.cpu().numpy()

mae = mean_absolute_error(y_test_cpu, predictions)
mse = mean_squared_error(y_test_cpu, predictions)
print(f'MAE: {mae:.4f}')
print(f'MSE: {mse:.4f}')
